In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('datasets/spotify/spotify_cleaned_dataset.csv')
df = df[['user_id__playlist_name','user_id','playlist_name','track_name', 'artist_name']]

In [ ]:
df['artist_name__track_name'] = df['artist_name'] + '__' + df['track_name']

In [ ]:
df['song_id'] = df.groupby('artist_name__track_name').grouper.group_info[0]

In [ ]:
songs = df[['artist_name', 'track_name', 'song_id']].copy()

In [ ]:
songs.set_index('song_id', inplace=True)
songs.index = songs.index.astype('str')
songs.drop_duplicates(inplace=True)
unknown_songs = songs[(songs['artist_name'].isin(['-', '', ' '])) | (songs['track_name'].isin(['-', '', ' ']))]
songs.drop(unknown_songs.index, inplace=True)
songs.reset_index(inplace=True)
songs.head()

In [ ]:
songs.shape

In [ ]:
playlists = df[['user_id__playlist_name','user_id', 'playlist_name', 'song_id']].copy()
playlists = playlists.groupby(['user_id__playlist_name','user_id', 'playlist_name'])['song_id'].apply(list).reset_index()

In [ ]:
playlist_data = [[str(song) for song in play if song not in unknown_songs.index] 
                 for play in playlists['song_id'].to_list()]

In [ ]:
len(playlist_data)

In [ ]:
with open('datasets/spotify/playlists.npy', 'wb') as f:
    np.save(f, np.array(playlist_data))

In [ ]:
songs.to_csv('datasets/spotify/songs.csv')